# GPT 架构详解 (Generative Pre-trained Transformer)

**SOTA 教育标准实现** | 大语言模型核心架构

---

## 学习目标

1. 理解GPT的Decoder-only架构设计
2. 掌握因果自注意力的数学原理和实现
3. 学会构建完整的GPT模型
4. 实现自回归文本生成

## 目录

1. [环境配置](#1-环境配置)
2. [GPT架构概述](#2-gpt架构概述)
3. [因果自注意力](#3-因果自注意力)
4. [GPT Block实现](#4-gpt-block实现)
5. [完整模型构建](#5-完整模型构建)
6. [验证测试](#6-验证测试)

---

## 1. 环境配置

In [ ]:
# 导入必要的库
import math
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F

# 设置随机种子确保结果可复现
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')
print(f'PyTorch版本: {torch.__version__}')

---

## 2. GPT架构概述

### 2.1 核心架构

GPT使用**Decoder-only Transformer**架构：

```
输入Token IDs
    ↓
Token Embedding + Position Embedding
    ↓
[GPT Block × N层]
    ├── LayerNorm → 因果自注意力 → 残差连接
    └── LayerNorm → FFN → 残差连接
    ↓
LayerNorm → LM Head → 输出概率
```

### 2.2 关键特点

| 特点 | 说明 |
|:-----|:-----|
| **因果注意力** | 只能看到当前及之前的token，防止信息泄漏 |
| **Pre-LN** | LayerNorm在注意力/FFN之前，训练更稳定 |
| **GELU激活** | 比ReLU更平滑，性能更好 |
| **权重绑定** | Token Embedding与LM Head共享权重 |

In [ ]:
@dataclass
class GPTConfig:
    """GPT模型配置类。
    
    使用dataclass管理所有超参数，避免魔术数字散落在代码中。
    
    Attributes:
        vocab_size: 词汇表大小
        max_seq_len: 最大序列长度
        n_layers: Transformer层数
        n_heads: 注意力头数
        d_model: 模型隐藏维度
        d_ff: FFN中间维度
        dropout: Dropout概率
    """
    vocab_size: int = 50257
    max_seq_len: int = 1024
    n_layers: int = 12
    n_heads: int = 12
    d_model: int = 768
    d_ff: int = None
    dropout: float = 0.1
    
    def __post_init__(self):
        # FFN中间维度默认为4倍d_model
        if self.d_ff is None:
            self.d_ff = 4 * self.d_model
        # 验证d_model能被n_heads整除
        assert self.d_model % self.n_heads == 0, \
            f'd_model({self.d_model})必须能被n_heads({self.n_heads})整除'

# 创建GPT-2 Small配置
config = GPTConfig()
print(f'GPT-2 Small配置: {config}')
print(f'每头维度: {config.d_model // config.n_heads}')

---

## 3. 因果自注意力

### 3.1 数学原理

因果注意力确保每个位置只能关注当前及之前的位置：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}} + M\right)V$$

其中因果掩码 $M$ 定义为：
$$M_{ij} = \begin{cases} 0 & \text{if } j \leq i \\ -\infty & \text{if } j > i \end{cases}$$

### 3.2 为什么需要因果掩码？

- **自回归生成**: 预测下一个token时，不能看到未来的token
- **防止信息泄漏**: 训练时确保模型只使用合法的上下文

In [ ]:
class CausalSelfAttention(nn.Module):
    """因果自注意力机制。
    
    核心思想:
        通过因果掩码确保每个位置只能关注当前及之前的位置，
        实现自回归语言建模。
    
    数学原理:
        Attention(Q,K,V) = softmax(QK^T/sqrt(d_k) + M) * V
        其中M是因果掩码（上三角为-inf）
    """
    
    def __init__(self, config: GPTConfig) -> None:
        super().__init__()
        self.n_heads = config.n_heads
        self.d_model = config.d_model
        self.d_k = config.d_model // config.n_heads
        
        # 合并的QKV投影（更高效）
        self.c_attn = nn.Linear(config.d_model, 3 * config.d_model)
        # 输出投影
        self.c_proj = nn.Linear(config.d_model, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        
        # 注册因果掩码（下三角矩阵）
        mask = torch.tril(torch.ones(config.max_seq_len, config.max_seq_len))
        self.register_buffer('mask', mask.view(1, 1, config.max_seq_len, config.max_seq_len))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """前向传播。
        
        Args:
            x: 输入张量 (batch, seq_len, d_model)
        Returns:
            输出张量 (batch, seq_len, d_model)
        """
        B, T, C = x.size()
        
        # Step 1: 计算Q, K, V
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.d_model, dim=2)
        
        # Step 2: 重塑为多头格式 (B, n_heads, T, d_k)
        q = q.view(B, T, self.n_heads, self.d_k).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.d_k).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.d_k).transpose(1, 2)
        
        # Step 3: 计算注意力分数并缩放
        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Step 4: 应用因果掩码
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
        
        # Step 5: Softmax归一化 + Dropout
        att = F.softmax(att, dim=-1)
        att = self.dropout(att)
        
        # Step 6: 加权求和
        y = att @ v
        
        # Step 7: 合并多头并投影
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        return self.c_proj(y)

In [ ]:
# 验证因果自注意力
def test_causal_attention() -> None:
    """验证因果自注意力的输出形状正确性。"""
    attn = CausalSelfAttention(config)
    x = torch.randn(2, 10, config.d_model)
    out = attn(x)
    
    assert out.shape == x.shape, f'输出形状错误: {out.shape}'
    print(f'✓ 输入形状: {x.shape}')
    print(f'✓ 输出形状: {out.shape}')
    print(f'✓ test_causal_attention 通过')

test_causal_attention()

---

## 4. GPT Block实现

### 4.1 Pre-LN结构

GPT Block使用Pre-LN（LayerNorm在子层之前）：

```
x → LN → Attention → + → LN → MLP → +
    └─────────────────┘   └──────────┘
         残差连接             残差连接
```

**Pre-LN vs Post-LN**:
- Pre-LN训练更稳定，不需要warmup
- Post-LN理论上性能更好，但训练困难

In [ ]:
class MLP(nn.Module):
    """GPT前馈网络。
    
    结构: Linear → GELU → Linear → Dropout
    """
    
    def __init__(self, config: GPTConfig) -> None:
        super().__init__()
        self.c_fc = nn.Linear(config.d_model, config.d_ff)
        self.c_proj = nn.Linear(config.d_ff, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.gelu(self.c_fc(x))
        return self.dropout(self.c_proj(x))


class GPTBlock(nn.Module):
    """GPT Transformer Block (Pre-LN)。"""
    
    def __init__(self, config: GPTConfig) -> None:
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.d_model)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.d_model)
        self.mlp = MLP(config)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Pre-LN: LayerNorm在子层之前
        x = x + self.attn(self.ln_1(x))  # 残差连接
        x = x + self.mlp(self.ln_2(x))   # 残差连接
        return x

In [ ]:
# 验证GPT Block
def test_gpt_block() -> None:
    """验证GPT Block的输出形状正确性。"""
    block = GPTBlock(config)
    x = torch.randn(2, 10, config.d_model)
    out = block(x)
    
    assert out.shape == x.shape, f'输出形状错误: {out.shape}'
    print(f'✓ GPT Block: {x.shape} -> {out.shape}')
    print(f'✓ test_gpt_block 通过')

test_gpt_block()

---

## 5. 完整模型构建

In [ ]:
class GPT(nn.Module):
    """GPT语言模型。
    
    完整的GPT架构，包含:
        - Token Embedding
        - Position Embedding
        - N × GPT Block
        - Layer Norm
        - LM Head (与Token Embedding权重绑定)
    """
    
    def __init__(self, config: GPTConfig) -> None:
        super().__init__()
        self.config = config
        
        # Embedding层
        self.wte = nn.Embedding(config.vocab_size, config.d_model)  # Token
        self.wpe = nn.Embedding(config.max_seq_len, config.d_model) # Position
        self.drop = nn.Dropout(config.dropout)
        
        # Transformer Blocks
        self.blocks = nn.ModuleList([GPTBlock(config) for _ in range(config.n_layers)])
        
        # 输出层
        self.ln_f = nn.LayerNorm(config.d_model)
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        
        # 权重绑定: Token Embedding与LM Head共享权重
        self.wte.weight = self.lm_head.weight
        
        # 统计参数量
        n_params = sum(p.numel() for p in self.parameters())
        print(f'GPT参数量: {n_params/1e6:.2f}M')
    
    def forward(self, idx, targets=None):
        """前向传播。
        
        Args:
            idx: 输入token ID (batch, seq_len)
            targets: 目标token ID，用于计算损失
        Returns:
            logits: 预测logits (batch, seq_len, vocab_size)
            loss: 交叉熵损失（如果提供targets）
        """
        B, T = idx.size()
        
        # 生成位置索引
        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)
        
        # Embedding: Token + Position
        x = self.drop(self.wte(idx) + self.wpe(pos))
        
        # 通过所有Transformer Block
        for block in self.blocks:
            x = block(x)
        
        # 输出层
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        # 计算损失
        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)), 
                targets.view(-1)
            )
        
        return logits, loss

In [ ]:
# 创建并验证模型
model = GPT(config)

# 测试前向传播
idx = torch.randint(0, config.vocab_size, (2, 10))
logits, _ = model(idx)
print(f'输入形状: {idx.shape}')
print(f'输出Logits形状: {logits.shape}')

---

## 6. 验证测试

In [ ]:
@torch.no_grad()
def generate(model, idx, max_new_tokens, temperature=1.0, top_k=None):
    """自回归文本生成。
    
    Args:
        model: GPT模型
        idx: 输入token ID (batch, seq_len)
        max_new_tokens: 最大生成token数
        temperature: 采样温度
        top_k: Top-K采样
    Returns:
        生成的token ID (batch, seq_len + max_new_tokens)
    """
    for _ in range(max_new_tokens):
        # 截断到最大长度
        idx_cond = idx if idx.size(1) <= model.config.max_seq_len \
            else idx[:, -model.config.max_seq_len:]
        
        # 前向传播
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperature
        
        # Top-K采样
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = float('-inf')
        
        # 采样下一个token
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    
    return idx

# 测试生成
start_ids = torch.randint(0, config.vocab_size, (1, 5))
generated = generate(model, start_ids, max_new_tokens=20, temperature=0.8, top_k=40)
print(f'输入长度: {start_ids.shape[1]}')
print(f'生成后长度: {generated.shape[1]}')
print(f'✓ 文本生成测试通过')

---

## 总结

### GPT核心组件

| 组件 | 作用 |
|:-----|:-----|
| **因果自注意力** | 确保自回归生成，防止信息泄漏 |
| **Pre-LN** | 训练更稳定，不需要warmup |
| **权重绑定** | 减少参数量，提升性能 |
| **GELU激活** | 更平滑的非线性，性能更好 |

### 进阶学习

- RoPE位置编码
- Flash Attention
- KV Cache优化